# Door Dash Case Study

## Part 1 Data Prep
### Library imports

In [191]:
import pandas as pd
import os
import numpy as np

In [27]:
df = pd.read_csv("../Data_Sources/moondash_data.csv") #using relative path to import data
df.head()

,effective_date,employee_id,job_function,location,job_level,employee_status,hire_date,pay_rate_type,role_volume_type
0,2022-03-31,16041,HR,"New York City, New York",L5,Active,2020-08-10,Salary,Low
1,2022-03-31,16449,Sales,"New York City, New York",L6,Active,2020-04-05,Salary,Low
2,2022-03-31,15822,Engineering,"San Francisco, California",L4,Active,2021-12-03,Salary,Low
3,2022-03-31,15417,Sales,"San Francisco, California",L5,Active,2021-08-01,Salary,Low
4,2022-03-31,14944,HR,"San Francisco, California",L4,Active,2021-03-01,Salary,Low


### Validating data

--no employee changes job function
--DATA GOES IN QUARTERS FROM 2022-03-31 TO 2024-31
--promotions denoted by seniority L2-L10
-----how to parse out promotion averages versus net new employees for roles
-----what roles are promotion only?
-----how to identify netnew 
-----average tenure of roles
-----seasonality of turnover (more around holidays etc)
---correlation between number of promotions + tenure
---pay type correlation
---correlation with current employee tenure


hypothesis:

pay_rate_type, job_function, location, tenure, & total promotions will impact turnover

how do we define turnover? for the year?

In [93]:
df[['effective_date','hire_date']] = df[['effective_date','hire_date']].apply(pd.to_datetime)

In [99]:
df.groupby("employee_id")['effective_date'].max().head()

employee_id
1   2022-06-30
2   2022-03-31
3   2022-06-30
4   2022-03-31
5   2022-03-31
Name: effective_date, dtype: datetime64[ns]

In [101]:
hire_caps = df.groupby("employee_id")['effective_date'].agg(['max','min'])
hire_role_caps = df.groupby(["employee_id",'job_function','job_level'])['effective_date'].agg(['max','min'])

In [210]:
df_a = pd.merge(df,hire_caps, how = "inner",on="employee_id").rename(columns={"max": "end_dt", "min": "start_dt"})
df_a2 = pd.merge(df_a,hire_role_caps, how = 'inner', on = ['employee_id','job_function','job_level']).rename(columns={"max": "end_role_dt", "min": "start_role_dt"})

In [212]:
# Creating number of promotions a given employee had
df_a2['rank'] = df_a2.groupby('employee_id')['start_role_dt'].rank(method='dense', ascending=False)

#many employees have a hire date before first effective date, we want to include their total tenure

df_a2['role_tenure'] = np.where(
    df_a2['rank'] == 1, #if first role, use hire date o/w use role start date
    (df_a2['effective_date'] - df_a2['hire_date']).dt.days, 
    (df_a2['effective_date'] - df_a2['start_role_dt']).dt.days 
)
# df_a2.head()
df_a2[df_a2['employee_id'] ==12].head()

,effective_date,employee_id,job_function,location,job_level,employee_status,hire_date,pay_rate_type,role_volume_type,end_dt,start_dt,end_role_dt,start_role_dt,rank,role_tenure
31767,2022-03-31,12,Strategy & Operations,"MoonMart 2, The Moon",L2,Active,2021-10-29,Hourly,High,2024-12-31,2022-03-31,2022-09-30,2022-03-31,2.0,0
31768,2022-06-30,12,Strategy & Operations,"MoonMart 2, The Moon",L2,Active,2021-10-29,Hourly,High,2024-12-31,2022-03-31,2022-09-30,2022-03-31,2.0,91
31769,2022-09-30,12,Strategy & Operations,"MoonMart 2, The Moon",L2,Active,2021-10-29,Hourly,High,2024-12-31,2022-03-31,2022-09-30,2022-03-31,2.0,183
31770,2022-12-31,12,Strategy & Operations,"MoonMart 2, The Moon",L3,Active,2021-10-29,Salary,Low,2024-12-31,2022-03-31,2024-12-31,2022-12-31,1.0,428
31771,2023-03-31,12,Strategy & Operations,"MoonMart 2, The Moon",L3,Active,2021-10-29,Salary,Low,2024-12-31,2022-03-31,2024-12-31,2022-12-31,1.0,518
